# Run: rdrp1 Human

```
Lead     : ababaian
Issue    : 
Version  : v0.4.0-dev : diamond-dev branch
start    : 2021 01 16
complete : 2021 01 xx
files    : ~/serratus/notebook/210108_ab/
s3_files : s3://serratus-public/notebook/210108_ab/
output   : s3://serratus-public/out/210116_rHu/
```

### Intro/Objectives

> The universe smiles upon you

### Human (non-controlled data)

- File: `hu_SraRunInfo.csv`

Search Term:
```
"Homo sapiens"[Organism] AND ("type_rnaseq"[Filter]) AND cluster_public[prop] AND "platform illumina"[Properties]
```
SRA Accessed: `2020/12/30`

Results: `680 939 / 789 931` (high error rate on batch 2 from NCBI)

**AND**

- File: `hu_meta_SraRunInfo.csv`

Search Term: 
```
"Homo sapiens"[Organism] AND ("metagenomic"[Filter] OR "metatranscriptomic"[Filter]) AND cluster_public[prop] AND "platform illumina"[Properties]
```
SRA Accessed: `2020/12/30`

Results: `47 763`

[See: 201230_sraRunInfo_Update.ipynb](./201230_sraRunInfo_Update.ipynb)


In [ ]:
## Fire up EC2 Instance
#sudo yum install -y docker
#sudo yum install -y git
#sudo service docker start

## Download latest serratus repo
#git clone -b diamond-dev https://github.com/ababaian/serratus.git; cd serratus/containers

## If you want to upload containers to your repository, include this.
#export DOCKERHUB_USER='serratusbio' # optional
#sudo docker login # optional
#
## Build all containers and upload them docker hub repo (if available)
#./build_containers.sh

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/210111_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/210111_ab'

# date and version
date
git rev-parse HEAD # commit version

Fri Jan 15 22:06:43 PST 2021
2ddf24166fdded0bc89a96b2541bcbc7442f98f9


In [3]:
cd $WORK

# Copy SRA Run Info
ln -s ../201230_sra/completed.sra ./
ln -s ../201230_sra/hu_SraRunInfo.csv ./
ln -s ../201230_sra/hu_meta_SraRunInfo.csv ./

md5sum hu*
ls -alh hu*

0e3424b9cbcb2249314dc1ce42fecf0c  hu_SraRunInfo.csv
ff4f6839d2434c5ecbb0bdc4cbbb990c  hu_meta_SraRunInfo.csv
lrwxrwxrwx 1 artem artem 31 Jan 15 22:25 hu_SraRunInfo.csv -> ../201230_sra/hu_SraRunInfo.csv
lrwxrwxrwx 1 artem artem 36 Jan 15 22:25 hu_meta_SraRunInfo.csv -> ../201230_sra/hu_meta_SraRunInfo.csv


In [4]:
cd $WORK

remove_done () {
  COMPLETE=$1
  SRA=$2
  
  # Sort unique SRA in the input RunInfo file
  cut -f1 -d',' $SRA \
    | sort - \
    > inputsra.tmp
  
  # Ensure completed SRA list is sorted
  sort $COMPLETE > complete.sort.tmp
  
  # Supress SRA-accessions only in file 1 or file 1+2
  # Show SRA unique to file 2
  # parse into a regex expression to find matching lines
  comm -1 -3 complete.sort.tmp inputsra.tmp \
    | sed 's/^$\n//g' - \
    | sed 's/^/^/g' - \
    | sed 's/$/,/g' \
    > incomplete.in.$SRA
    
  grep -f incomplete.in.$SRA $SRA > undone_$SRA
  
  rm *.tmp
  # Still crashing due to high memory
  # but will give a list of "undone" accessions
}

remove_done completed.sra hu_meta_SraRunInfo.csv

In [5]:
# Chop up file for batch-runs
sed -n '1,275000p;275001q' hu_SraRunInfo.csv > hu_SraRunInfo_1.csv

head -n1 hu_SraRunInfo.csv > hu_SraRunInfo_2.csv
sed -n '275000,550000p;550001q' hu_SraRunInfo.csv >> hu_SraRunInfo_2.csv

head -n1 hu_SraRunInfo.csv > hu_SraRunInfo_3.csv
tail -n+2 undone_hu_meta_SraRunInfo.csv >> hu_SraRunInfo_3.csv
tail -n+550001 hu_SraRunInfo.csv >> hu_SraRunInfo_3.csv

In [ ]:
#aws s3 sync ./ $S3_WORK/

### Terraform Initialize

In [ ]:
## UPDATE serratus/terraform/main/main.tf
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

In [16]:
# BATCH 1
#
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

Connection to ec2-3-227-153-2.compute-1.amazonaws.com closed by remote host.
Connection to ec2-3-208-167-87.compute-1.amazonaws.com closed by remote host.
diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..f0a7962 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
- 

+  filename        = "${path.module}/create_tunnels.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic up

module.align.aws_iam_role_policy.ec2Terminate: Creating...
module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creating...
module.download.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 1s [id=profile-serratus-dl]
module.merge.module.iam_role.aws_iam_instance_profile.profile: Creating...
module.scheduler.module.ecs_cluster.aws_iam_instance_profile.p: Creation complete after 2s [id=instance-profile-serratus-scheduler]
module.align.aws_iam_role_policy.AdjustAutoScaling: Creation complete after 1s [id=SerratusIamRole-serratus-align:AdjustAutoScaling-serratus-align]
module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creating...
module.monitoring.module.ecs_cluster.aws_iam_role_policy_attachment.instance_attachment: Creating...
module.align.aws_iam_role_policy.ec2Describe: Creation complete after 1s [id=SerratusIamRole-serratus-align:DescribeEC2Instances-serratus-align]
module.schedule

module.scheduler.module.ecs_cluster.aws_instance.i: Still creating... [10s elapsed]
module.monitoring.module.ecs_cluster.aws_instance.i: Still creating... [10s elapsed]
module.scheduler.module.ecs_cluster.aws_instance.i: Creation complete after 17s [id=i-0ecbd76c6cac92d8b]
module.monitoring.module.ecs_cluster.aws_instance.i: Creation complete after 17s [id=i-0a22382c418503c09]
module.scheduler.aws_eip.sch: Creating...
module.monitoring.aws_eip.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-047792ca6225a1fd7]
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-0a2daa

In [37]:
# BATCH 2
#
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..f0a7962 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,68 +89,73 @@ re

     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corres

module.monitoring.aws_iam_role_policy_attachment.attachment: Creating...
module.download.aws_cloudwatch_log_group.g: Creation complete after 1s [id=serratus-dl]
module.download.module.iam_role.aws_iam_role_policy.cloudwatch: Creating...
module.align.aws_iam_role_policy.ec2Describe: Creation complete after 1s [id=SerratusIamRole-serratus-align:DescribeEC2Instances-serratus-align]
module.download.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creating...
module.align.aws_iam_role_policy.AdjustAutoScaling: Creation complete after 0s [id=SerratusIamRole-serratus-align:AdjustAutoScaling-serratus-align]
module.download.module.iam_role.aws_iam_instance_profile.profile: Creating...
module.monitoring.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-monitor:CloudwatchGetMetrics]
module.scheduler.aws_iam_role_policy.scheduler: Creation complete after 1s [id=SerratusIamRole-scheduler:DescribeInstances-sched

module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-07372917af15a5f80]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-033a9006be61383c1]
local_file.create_tunnel: Creating...
local_file.hosts: Creating...
local_file.create_tunnel: Creation complete after 0s [id=4c78b9208fb8c80569f33e51d20234b4b631da2b]
module.merge.aws_launch_configuration.worker: Creating...
module.download.aws_launch_configuration.worker: Creating...
local_file.hosts: Creation complete after 0s [id=3b267fea8acb8e51c2debad0f3dbc55396edbd32]
module.align.aws_launch_configuration.worker: Creating...
module.align.aws_launch_configuration.worker: Creation complete after 3s [id=serratu

In [6]:
# BATCH 3
#
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..f0a7962 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,68 +89,73 @@ re

     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corres

module.download.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 1s [id=profile-serratus-dl]
module.download.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-dl:CloudWatchLogsCreate-serratus-dl]
module.monitoring.module.ecs_cluster.aws_iam_role_policy_attachment.instance_attachment: Creation complete after 1s [id=SerratusEcsInstanceRole-monitor-20210116063241413800000005]
module.align.aws_iam_role_policy.ec2Describe: Creating...
module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creating...
module.align.aws_iam_role_policy.ec2Terminate: Creation complete after 1s [id=SerratusIamRole-serratus-align:TerminateEC2Instances-serratus-align]
module.merge.module.iam_role.aws_iam_instance_profile.profile: Creating...
module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creating...
module.align.aws_iam_role_policy.AdjustAutoScaling: Creati

module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 0s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-01fb2831473d7b8f1]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-01b72f1985791702e]
module.download.aws_launch_configuration.worker: Creating...
local_file.hosts: Creating...
module.align.aws_launch_configuration.worker: Creating...
local_file.hosts: Creation complete after 0s [id=f003ecb372a775a21b6a784f94744981a3dfc3ba]
module.merge.aws_launch_configuration.worker: Creating...
local_file.create_tunnel: Creating...
local_file.create_tunnel: Creation complete after 0s [id=d3d80e5f6801109c9c713cc2ca205e58f1dd5b3a]
module.merge.aws_laun

Serratus Config backup 
```
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 40,
  "ALIGN_SCALING_CONSTANT": 0.08,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 3200,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 1350,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 25,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}
```

In [38]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>

Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [22]:
# BATCH 1
cd $WORK
BATCH1='hu_SraRunInfo_1.csv'
wc -l $WORK/$BATCH1

275000 /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_1.csv


In [23]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH1

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_1.csv
  date: Sat Jan 16 08:05:45 PST 2021
  wc  : 275000 /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_1.csv
  md5 : 671d6bfed11c23d01855e235f47d0086  /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_1.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
54056effaadf0ecf44c47bb3a7b6b029  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
86b0d5a6514d50508eb0c5c82d207508  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
51af0c87239dc949c8bd75c8ad2e7395  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
44d01dcf464292f42eb63df0b7621577  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":100

In [40]:
# BATCH 2
cd $WORK
BATCH2='hu_SraRunInfo_2.csv'
wc -l $WORK/$BATCH2

275002 /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_2.csv


In [41]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH2

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_2.csv
  date: Sat Jan 16 19:07:05 PST 2021
  wc  : 275002 /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_2.csv
  md5 : af83cfea19cdc4357cc8ba3fd60d9259  /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_2.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
c640e22ac558ea34aa680433f5b75905  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
7d7580c852999d42c85071b7c8a88be3  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
e643dfb27e6ec3c6b672471270961ddd  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
d334a806a729f9a7171a9e8232b83a5a  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":100

In [11]:
# BATCH 3
cd $WORK
BATCH3='hu_SraRunInfo_3.csv'
wc -l $WORK/$BATCH3

269022 /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_3.csv


In [13]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH3

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_3.csv
  date: Fri Jan 15 22:37:27 PST 2021
  wc  : 269022 /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_3.csv
  md5 : 608a009d711765b4f144a565ccddbbb9  /home/artem/serratus/notebook/210111_ab/hu_SraRunInfo_3.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
fa69308ffe66e3d3848b0c7bdd0b5808  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
665870eaab7b517b96ae1591b41b062a  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
e22a79c9da9a73629e14ff546b50cbde  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
4fe41df7a28a04df78de3bb3c28e70aa  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":100

## Run Serratus

In [49]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -sT serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 35,
  "ALIGN_SCALING_CONSTANT": 0.08,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 0,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 0,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 200,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":35,"ALIGN_SCALING_CONSTANT":0.08,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":0,"CLEAR_INTERVAL":999999,"DL_ARGS":"","DL_MAX_INCREASE":20,"DL_SCALING_CONSTANT":0.1,"DL_SCALING_ENABLE":true,"DL_SCALING_MAX":0,"GENOME":"rdrp1","MERGE_ARGS":"protein","MERGE_MAX_INCREASE":25,"MERGE_SCALING_CONSTANT":0.1,"MERGE_SCALING_ENABLE":true,"MERGE_SCALING_MAX":200,"SCALING_INT

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Regular Shutdown
cd $TF
terraform destroy

## Nuke Shutdown
#cd $TF
#
#aws ec2 describe-instances \
#  --filter Name=tag:Name,Values=serratus-align-instance \
#  > align_instances.json
#
#jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
#  | pv -l \
#  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

# Notes: 



psummary: `https://serratus-public.s3.amazonaws.com/out/210108_r1p/psummary/<SRA>.psummary`

SRA site: `https://www.ncbi.nlm.nih.gov/sra/?term=<SRA>`
